# Выдача охватов по списку

In [1]:
import load_env

In [2]:
import asyncio
import datetime as dt
import logging

import pandas as pd
from tqdm.auto import tqdm

from scanner import Scanner
from utils import get_nicknames
import supabasefs
import supabase
import os


logging.getLogger("pyrogram").setLevel("ERROR")
logging.getLogger("urllib3").setLevel("ERROR")

In [3]:
LIMIT_HISTORY = dt.timedelta(days=30)  # насколько лезть вглубь чата

In [4]:
client = supabase.create_client(os.environ["SUPABASE_URL"], os.environ["SUPABASE_KEY"])
fs = supabasefs.SupabaseTableFileSystem(client, "sessions")

In [5]:
scanner = Scanner(fs=fs, chat_cache=False)


## Взять список каналов

In [6]:
# channels = {"@bekirovdanil", "@flipping_invest", "@atsogoev", "@ligainvestblog", "@d_smirnovv", "@thebestinvestru"}

text = (
"""https://t.me/homestagersecret
https://t.me/alexdyachenko
https://t.me/nina_homestage
https://t.me/na_remonte
https://t.me/Soboleva_Viktory
https://t.me/stirmanova
https://t.me/holly_design
https://t.me/+OJX_z3RxzAthNWQy?fbclid=PAAaYRLkFKfdnGnFeji_FX0zThHbJJSvxPFv-PAJtNBEm4kYXcLmasde1MRBk
https://t.me/profi_homestaging
https://t.me/nana_eee
https://t.me/panchukova
https://t.me/holding_osnova
https://t.me/Semenenko_Dmitriy
https://t.me/homestaging_key
https://t.me/novostroykithebest

https://t.me/aktivo_news
https://t.me/kolesnik_company
https://t.me/iakushechkin
https://t.me/RentonYard
https://t.me/investorium_realty
https://t.me/ottmaxim
https://t.me/gosman_development



https://t.me/+909J7WDWTcJjN2My


https://t.me/Pro_smarent
https://t.me/investmoney128
https://t.me/stone_by_stonehedge
https://t.me/pantya_yuliu

https://t.me/polonsky
https://t.me/sybarenda
https://t.me/simpleestate



https://t.me/kachanovnik

https://t.me/million_na_nedvijke

https://t.me/soboleva_invest

нет
https://t.me/ligainvestblog
https://t.me/anna_finance_advanced
https://t.me/dohod_s_nedvizki
https://t.me/gordashnikovaland
https://t.me/pereplanirovkagonenko
https://t.me/marinova_blog
https://t.me/ruslan_rentaved
https://t.me/Capital_Glovatskiy
https://t.me/flippingkurs
https://t.me/shulick
https://t.me/flipping_todayprice
https://t.me/ceo2pillowschannel
https://t.me/ddominvest""")

channels_original = text.split("\n")
lines_dict = {}
for i, line in enumerate(channels_original):
    usernames = get_nicknames(line)
    if len(usernames) == 1:
        lines_dict[i] = next(iter(usernames))
    elif len(usernames) == 0:
        continue
    else:
        raise ValueError(len(usernames))
    
channels = set(lines_dict.values())

## Взять статистику по каждому каналу

In [7]:
async def collect_stats(channel) -> int:
    reaches = [
        msg.views or 0
        async for msg in scanner.get_chat_history(
            channel, min_date=dt.datetime.now() - LIMIT_HISTORY
        )
    ]

    return channel, sum(reaches) // len(reaches) if reaches else 0


In [8]:
results = []

with tqdm(total=len(channels)) as pbar:
    async with scanner.session(pbar):
        tasks = [asyncio.create_task(collect_stats(channel)) for channel in channels]
        for finished_task in asyncio.as_completed(tasks):
            results.append(await finished_task)
            pbar.update()

  0%|          | 0/44 [00:00<?, ?it/s]

## Выдать охват

In [9]:
def calc_stats(channel_list):
    stats = pd.DataFrame(channel_list, columns=["username", "reach"])
    return stats.sort_values("reach", ascending=False)

In [10]:
stats = calc_stats(results)
stats

,username,reach
35,@stone_by_stonehedge,5438
39,@soboleva_viktory,4758
36,@anna_finance_advanced,4722
26,@pro_smarent,3514
32,@rentonyard,2596
34,@aktivo_news,2418
12,@simpleestate,2412
38,@marinova_blog,2281
43,@holding_osnova,2274
33,@na_remonte,1892


In [11]:
total_reach = stats.reach.sum()
total_reach

58304

In [12]:
final_lines = {
    i: str(stats.reach[stats.username == lines_dict[i]].values[0])
    for i in lines_dict
}
pd.DataFrame(
    final_lines.get(i, None) for i in range(len(channels_original))
).to_clipboard(index=False)